<a href="https://colab.research.google.com/github/meizhong986/WhisperJAV/blob/main/WhisperWithVAD_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WhisperWithVAD PRO


---


This is a fork of the original WhisperWithVAD

This version exposes new options in recent releases of Whisper + hyperparameter settings


```
**Changelog**
*   2024-03-06: Forked from the maintenance release
```



In [ ]:
#@title Whisper Transcription Parameters


model_size = "large-v2"  # @param ["large-v3", "large-v2", "medium", "large"]
language = "japanese"  # @param {type:"string"}
translation_mode = "End-to-end Whisper (default)"  # @param ["End-to-end Whisper (default)", "Whisper -> DeepL", "No translation"]

# @markdown VAD settings and DeepL:
deepl_authkey = ""  # @param {type:"string"}
source_separation = False  # @param {type:"boolean"}
vad_threshold = 0.4  # @param {type:"number"}
chunk_threshold = 3.0  # @param {type:"number"}
deepl_target_lang = "EN-US"  # @param {type:"string"}
max_attempts = 1  # @param {type:"integer"}


#@markdown Enter the values for the transcriber parameters. Leave unchanged if not sure.

verbose = False #@param {type:"boolean"}
temperature_input = "0.0" #@param {type:"string"}
compression_ratio_threshold = 2.4 #@param {type:"number"}
logprob_threshold = -1.0 #@param {type:"number"}
no_speech_threshold = 0.6 #@param {type:"number"}
condition_on_previous_text = False #@param {type:"boolean"}
initial_prompt = "" #@param {type:"string"}
word_timestamps = True #@param {type:"boolean"}
clip_timestamps_input = "0" #@param {type:"string"}
hallucination_silence_threshold = 2.0 #@param {type:"number"}


#@markdown Decoding Options (for advanced configurations, leave unchnaged if unsure):
best_of = 2 #@param {type:"number"}
beam_size = 2 #@param {type:"number"}
patience = 1 #@param {type:"number"}
length_penalty = "" #@param {type:"string"}
prefix = "" #@param {type:"string"}
suppress_tokens = "-1" #@param {type:"string"}
suppress_blank = True #@param {type:"boolean"}
without_timestamps = False #@param {type:"boolean"}
max_initial_timestamp = 1.0 #@param {type:"number"}
fp16 = True #@param {type:"boolean"}

# Parsing and converting form inputs
try:
    temperature = tuple(float(temp.strip()) for temp in temperature_input.split(',')) if ',' in temperature_input else float(temperature_input)
except ValueError:
    temperature = (0.0, 0.2, 0.4, 0.6, 0.8, 1.0)  # Default

clip_timestamps = clip_timestamps_input.split(',') if ',' in clip_timestamps_input else clip_timestamps_input
if clip_timestamps != "0":
    try:
        clip_timestamps = list(map(float, clip_timestamps)) if isinstance(clip_timestamps, list) else float(clip_timestamps)
    except ValueError:
        clip_timestamps = "0"  # Default if parsing fails

language = None if not language else language
initial_prompt = None if initial_prompt == "" else initial_prompt
length_penalty = None if length_penalty == "" else float(length_penalty)


assert max_attempts >= 1
assert vad_threshold >= 0.01
assert chunk_threshold >= 0.1
assert language != ""
if translation_mode == "End-to-end Whisper (default)":
    task = "translate"
    run_deepl = False
elif translation_mode == "Whisper -> DeepL":
    task = "transcribe"
    run_deepl = True
elif translation_mode == "No translation":
    task = "transcribe"
    run_deepl = False
else:
    raise ValueError("Invalid translation mode")



# Prepare transcription options
transcription_options = {
    "verbose": verbose,
    "compression_ratio_threshold": compression_ratio_threshold,
    "logprob_threshold": logprob_threshold,
    "no_speech_threshold": no_speech_threshold,
    "condition_on_previous_text": condition_on_previous_text,
    "initial_prompt": initial_prompt,
    "word_timestamps": word_timestamps,
    "clip_timestamps": clip_timestamps,
    "hallucination_silence_threshold": hallucination_silence_threshold
}

# Prepare decoding options
decoding_options = {
    "task": task,
    "language": language,
    "temperature": temperature,
    "best_of": best_of,
    "beam_size": beam_size,
    "patience": patience,
    "length_penalty": length_penalty,
    "prefix": prefix,
    "suppress_tokens": suppress_tokens,
    "suppress_blank": suppress_blank,
    "without_timestamps": without_timestamps,
    "max_initial_timestamp": max_initial_timestamp,
    "fp16": fp16,
}





In [ ]:
#@markdown **GPU check** (you typically want a V100, P100 or T4)
!nvidia-smi -L
!nvidia-smi

In [ ]:
#@markdown **Mount Google Drive** (skip this if your audio file isn't stored there)
from google.colab import drive
drive.mount('drive')

In [ ]:
#@markdown **Setup Whisper  -grab yourself a coffee, this may take about 3 minutes**
%%capture
!apt-get install sox libsox-fmt-mp3 libsndfile1 ffmpeg
!pip install deepl srt ffmpeg-python spleeter
!pip uninstall -y openai-whisper openai-whisper-20230918  torch torchvision torchaudio torchtext torchdata
!pip install cohere openai
!pip install --no-cache-dir  torch==1.12.1 torchvision torchaudio torchtext torchdata triton==2.0.0 tiktoken==0.3.3
!pip install --no-cache-dir  --no-deps --force-reinstall git+https://github.com/openai/whisper.git

print("Done")

In [ ]:
#@markdown **Upload audio file to Colab** <br>
#@markdown If this step fails, or is very slow, try one of these options:
#@markdown * Drag your file into the Files sidebar, and set audio_path to the filename
#@markdown * OR upload it to Google Drive, mount it, and set audio_path to the absolute path
#@markdown * OR upload it to a service like Litterbox, and set audio_path to the URL
from google.colab import files
files = files.upload()
if len(files) > 0:
    uploaded_file = list(files)[0]
    print("Upload complete")


In [ ]:
#@markdown **Run Whisper**

# @markdown Required settings:
audio_path = "/content/drive/MyDrive/test.wav"  # @param {type:"string"}
assert audio_path != ""


import tensorflow as tf
import torch
import whisper
import os
import ffmpeg
import srt
from tqdm import tqdm
import datetime
import deepl
import urllib.request
import json
from google.colab import files


if "http://" in audio_path or "https://" in audio_path:
    print("Downloading audio...")
    urllib.request.urlretrieve(audio_path, "input_file")
    audio_path = "input_file"
else:
    if not os.path.exists(audio_path):
        try:
            audio_path = uploaded_file
            if not os.path.exists(audio_path):
                raise ValueError("Input audio not found. Is your audio_path correct?")
        except NameError:
            raise ValueError("Input audio not found. Did you upload a file?")

out_path = os.path.splitext(audio_path)[0] + ".srt"
out_path_pre = os.path.splitext(audio_path)[0] + "_Untranslated.srt"
if source_separation:
    print("Separating vocals...")
    !ffprobe -i "{audio_path}" -show_entries format=duration -v quiet -of csv="p=0" > input_length
    with open("input_length") as f:
        input_length = int(float(f.read())) + 1
    !spleeter separate -d {input_length} -p spleeter:2stems -o output "{audio_path}"
    spleeter_dir = os.path.basename(os.path.splitext(audio_path)[0])
    audio_path = "output/" + spleeter_dir + "/vocals.wav"

print("Encoding audio...")
if not os.path.exists("vad_chunks"):
    os.mkdir("vad_chunks")
ffmpeg.input(audio_path).output(
    "vad_chunks/silero_temp.wav",
    ar="16000",
    ac="1",
    acodec="pcm_s16le",
    map_metadata="-1",
    fflags="+bitexact",
).overwrite_output().run(quiet=True)

print("Running VAD...")
model, utils = torch.hub.load(
    repo_or_dir="snakers4/silero-vad", model="silero_vad", onnx=False
)

(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils

# Generate VAD timestamps
VAD_SR = 16000
wav = read_audio("vad_chunks/silero_temp.wav", sampling_rate=VAD_SR)
t = get_speech_timestamps(wav, model, sampling_rate=VAD_SR, threshold=vad_threshold)

# Add a bit of padding, and remove small gaps
for i in range(len(t)):
    t[i]["start"] = max(0, t[i]["start"] - 3200)  # 0.2s head
    t[i]["end"] = min(wav.shape[0] - 16, t[i]["end"] + 20800)  # 1.3s tail
    if i > 0 and t[i]["start"] < t[i - 1]["end"]:
        t[i]["start"] = t[i - 1]["end"]  # Remove overlap

# If breaks are longer than chunk_threshold seconds, split into a new audio file
# This'll effectively turn long transcriptions into many shorter ones
u = [[]]
for i in range(len(t)):
    if i > 0 and t[i]["start"] > t[i - 1]["end"] + (chunk_threshold * VAD_SR):
        u.append([])
    u[-1].append(t[i])

# Merge speech chunks
for i in range(len(u)):
    save_audio(
        "vad_chunks/" + str(i) + ".wav",
        collect_chunks(u[i], wav),
        sampling_rate=VAD_SR,
    )
os.remove("vad_chunks/silero_temp.wav")

# Convert timestamps to seconds
for i in range(len(u)):
    time = 0.0
    offset = 0.0
    for j in range(len(u[i])):
        u[i][j]["start"] /= VAD_SR
        u[i][j]["end"] /= VAD_SR
        u[i][j]["chunk_start"] = time
        time += u[i][j]["end"] - u[i][j]["start"]
        u[i][j]["chunk_end"] = time
        if j == 0:
            offset += u[i][j]["start"]
        else:
            offset += u[i][j]["start"] - u[i][j - 1]["end"]
        u[i][j]["offset"] = offset

# Run Whisper on each audio chunk
print("Running Whisper...")
model = whisper.load_model(model_size)
subs = []
segment_info = []
sub_index = 1
suppress_low = [
    "Thank you",
    "Thanks for",
    "ike and ",
    "Bye.",
    "Bye!",
    "Bye bye!",
    "lease sub",
    "The end.",
    "視聴",
]
suppress_high = [
    "ubscribe",
    "my channel",
    "the channel",
    "our channel",
    "ollow me on",
    "for watching",
    "hank you for watching",
    "for your viewing",
    "r viewing",
    "Amara",
    "next video",
    "full video",
    "ranslation by",
    "ranslated by",
    "ee you next week",
    "ご視聴",
    "視聴ありがとうございました",
]
for i in tqdm(range(len(u))):
    line_buffer = []  # Used for DeepL
    for x in range(max_attempts):
        result = model.transcribe(
            "vad_chunks/" + str(i) + ".wav",
          **transcription_options,
          **decoding_options
        )



        # Break if result doesn't end with severe hallucinations
        if len(result["segments"]) == 0:
            break
        elif result["segments"][-1]["end"] < u[i][-1]["chunk_end"] + 10.0:
            break
        elif x+1 < max_attempts:
            print("Retrying chunk", i)
    for r in result["segments"]:
        # Skip audio timestamped after the chunk has ended
        if r["start"] > u[i][-1]["chunk_end"]:
            continue
        # Reduce log probability for certain words/phrases
        for s in suppress_low:
            if s in r["text"]:
                r["avg_logprob"] -= 0.15
        for s in suppress_high:
            if s in r["text"]:
                r["avg_logprob"] -= 0.35
        # Keep segment info for debugging
        del r["tokens"]
        segment_info.append(r)
        # Skip if log prob is low or no speech prob is high
        if r["avg_logprob"] < -1.0 or r["no_speech_prob"] > 0.7:
            continue
        # Set start timestamp
        start = r["start"] + u[i][0]["offset"]
        for j in range(len(u[i])):
            if (
                r["start"] >= u[i][j]["chunk_start"]
                and r["start"] <= u[i][j]["chunk_end"]
            ):
                start = r["start"] + u[i][j]["offset"]
                break
        # Prevent overlapping subs
        if len(subs) > 0:
            last_end = datetime.timedelta.total_seconds(subs[-1].end)
            if last_end > start:
                subs[-1].end = datetime.timedelta(seconds=start)
        # Set end timestamp
        end = u[i][-1]["end"] + 0.5
        for j in range(len(u[i])):
            if r["end"] >= u[i][j]["chunk_start"] and r["end"] <= u[i][j]["chunk_end"]:
                end = r["end"] + u[i][j]["offset"]
                break
        # Add to SRT list
        subs.append(
            srt.Subtitle(
                index=sub_index,
                start=datetime.timedelta(seconds=start),
                end=datetime.timedelta(seconds=end),
                content=r["text"].strip(),
            )
        )
        sub_index += 1

with open("segment_info.json", "w", encoding="utf8") as f:
    json.dump(segment_info, f, indent=4)

# DeepL translation
translate_error = False
if run_deepl:
    print("Translating...")
    with open(out_path_pre, "w", encoding="utf8") as f:
        f.write(srt.compose(subs))
    print("(Untranslated subs saved to", out_path_pre, ")")

    lines = []
    punct_match = ["。", "、", ",", ".", "〜", "！", "!", "？", "?", "-"]
    for i in range(len(subs)):
        if language.lower() == "japanese":
            if subs[i].content[-1] not in punct_match:
                subs[i].content += "。"
            subs[i].content = "「" + subs[i].content + "」"
        else:
            if subs[i].content[-1] not in punct_match:
                subs[i].content += "."
            subs[i].content = '"' + subs[i].content + '"'
    for i in range(len(subs)):
        lines.append(subs[i].content)

    grouped_lines = []
    english_lines = []
    for i, l in enumerate(lines):
        if i % 30 == 0:
            # Split lines into smaller groups, to prevent error 413
            grouped_lines.append([])
            if i != 0:
                # Include previous 3 lines, to preserve context between splits
                grouped_lines[-1].extend(grouped_lines[-2][-3:])
        grouped_lines[-1].append(l.strip())

    try:
        translator = deepl.Translator(deepl_authkey)
        for i, n in enumerate(tqdm(grouped_lines)):
            x = ["\n".join(n).strip()]
            if language.lower() == "japanese":
                result = translator.translate_text(x, source_lang="JA", target_lang=deepl_target_lang)
            else:
                result = translator.translate_text(x, target_lang=deepl_target_lang)
            english_tl = result[0].text.strip().splitlines()
            assert len(english_tl) == len(n), (
                "Invalid translation line count ("
                + str(len(english_tl))
                + " vs "
                + str(len(n))
                + ")"
            )
            if i != 0:
                english_tl = english_tl[3:]
            remove_quotes = dict.fromkeys(map(ord, '"„“‟”＂「」'), None)
            for e in english_tl:
                english_lines.append(
                    e.strip().translate(remove_quotes).replace("’", "'")
                )
        for i, e in enumerate(english_lines):
            subs[i].content = e
    except Exception as e:
        print("DeepL translation error:", e)
        print("(downloading untranslated version instead)")
        translate_error = True

# Write SRT file
if translate_error:
    files.download(out_path_pre)
else:
    # Removal of garbage lines
    garbage_list = [
        "a",
        "aa",
        "ah",
        "ahh",
        "ha",
        "haa",
        "hah",
        "haha",
        "hahaha",
        "mmm",
        "mm",
        "m",
        "h",
        "o",
        "mh",
        "mmh",
        "hm",
        "hmm",
        "huh",
        "oh",
    ]
    need_context_lines = [
        "feelsgod",
        "godbye",
        "godnight",
        "thankyou",
    ]
    clean_subs = list()
    last_line_garbage = False
    for i in range(len(subs)):
        c = subs[i].content
        c = (
            c.replace(".", "")
            .replace(",", "")
            .replace(":", "")
            .replace(";", "")
            .replace("!", "")
            .replace("?", "")
            .replace("-", " ")
            .replace("  ", " ")
            .replace("  ", " ")
            .replace("  ", " ")
            .lower()
            .replace("that feels", "feels")
            .replace("it feels", "feels")
            .replace("feels good", "feelsgood")
            .replace("good bye", "goodbye")
            .replace("good night", "goodnight")
            .replace("thank you", "thankyou")
            .replace("aaaaaa", "a")
            .replace("aaaa", "a")
            .replace("aa", "a")
            .replace("aa", "a")
            .replace("mmmmmm", "m")
            .replace("mmmm", "m")
            .replace("mm", "m")
            .replace("mm", "m")
            .replace("hhhhhh", "h")
            .replace("hhhh", "h")
            .replace("hh", "h")
            .replace("hh", "h")
            .replace("oooooo", "o")
            .replace("oooo", "o")
            .replace("oo", "o")
            .replace("oo", "o")
        )
        is_garbage = True
        for w in c.split(" "):
            if w.strip() == "":
                continue
            if w.strip() in garbage_list:
                continue
            elif w.strip() in need_context_lines and last_line_garbage:
                continue
            else:
                is_garbage = False
                break
        if not is_garbage:
            clean_subs.append(subs[i])
        last_line_garbage = is_garbage
    with open(out_path, "w", encoding="utf8") as f:
        f.write(srt.compose(clean_subs))
    print("\nDone! Subs written to", out_path)
    print("Downloading SRT file:")
    files.download(out_path)




---



***# Guide / Notes / Credits***



---





Settings for large-v2 higer quality output

(Note 1: it increases the execution time)

(Note 2: large-v3 may need different settings)

> temperature= 0.0,0.2,0.4

> compression_ratio_threshold = 2.4

> logprob_threshold = -1.0

> no_speech_threshold = 0.6

> clip_timestamps = 0

> hallucination_silence_threshold = 2.0

> best_of = 6

> beam_size = 6

> patience = 3

> max_initial_timestamp = 1.0









